In [1]:
%pip install pandas 
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np



In [4]:
stavbe2019 = pd.read_csv(r"clean_GURS\new_prodaja2019.csv", sep=',', encoding='utf-8')
stavbe2019.sample(10)

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,LETO,cena,novogradnja
2594,MOSTE,LJUBLJANA,1961,9.0,9.0,2019,14500.0,NaN
658,RUDNIK,LJUBLJANA,1950,40.0,40.0,2019,60000.0,NaN
1448,POLJANSKO PREDMESTJE,LJUBLJANA,1964,61.9,50.1,2019,155000.0,NaN
1753,STOŽICE,LJUBLJANA,1978,53.6,51.3,2019,125000.0,NaN
1345,ZGORNJA ŠIŠKA,LJUBLJANA,1967,30.4,27.0,2019,95000.0,NaN
983,STOŽICE,LJUBLJANA,1976,94.6,90.7,2019,102500.0,NaN
2055,BEŽIGRAD,LJUBLJANA,2019,32.8,28.5,2019,146063.0,1.0
2823,ŠENTVID NAD LJUBLJANO,LJUBLJANA,1986,86.3,70.2,2019,175000.0,NaN
2115,TABOR,LJUBLJANA,2018,123.4,100.2,2019,339000.0,1.0
29,KARLOVŠKO PREDMESTJE,LJUBLJANA,2004,71.4,57.8,2019,283000.0,NaN


In [5]:
stavbe2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3137 entries, 0 to 3136
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   ime_ko             3137 non-null   object 
 1   naselje            3127 non-null   object 
 2   letnica_izgradnje  3137 non-null   int64  
 3   m2_bruto           3137 non-null   float64
 4   m2_neto            3137 non-null   float64
 5   LETO               3137 non-null   int64  
 6   cena               3137 non-null   float64
 7   novogradnja        328 non-null    float64
dtypes: float64(4), int64(2), object(2)
memory usage: 196.2+ KB


In [6]:
#preparing the year column for Power BI
stavbe2019['LETO'] = pd.to_datetime("2019-01-01")

In [7]:

# creating a boolean column 'NOVOGRADNJA_1' based on 'NOVOGRADNJA'
stavbe2019['novogradnja_1'] = stavbe2019['novogradnja'] == 1

# Preview the new column
stavbe2019[['novogradnja', 'novogradnja_1']].head(10)


,novogradnja,novogradnja_1
0,1.0,True
1,1.0,True
2,NaN,False
3,NaN,False
4,NaN,False
5,NaN,False
6,NaN,False
7,NaN,False
8,NaN,False
9,NaN,False


In [12]:
#Printing the number of rows with missing values in 'm2_neto' column
stavbe2019[stavbe2019['m2_neto'].isna()].count()

ime_ko               0
naselje              0
letnica_izgradnje    0
m2_bruto             0
m2_neto              0
LETO                 0
cena                 0
novogradnja          0
novogradnja_1        0
dtype: int64

In [9]:
stavbe2019[stavbe2019['m2_neto'] == 0].count()

ime_ko               48
naselje              47
letnica_izgradnje    48
m2_bruto             48
m2_neto              48
LETO                 48
cena                 48
novogradnja           9
novogradnja_1        48
dtype: int64

In [10]:
stavbe2019.loc[stavbe2019['m2_neto'] == 0, 'm2_neto'] = np.nan

In [11]:
stavbe2019.fillna({'m2_neto': stavbe2019['m2_bruto']}, inplace=True)

In [13]:
def oceni_sobnost(m2):
    if pd.isna(m2):
        return None
    if m2 < 35:
        return 1
    elif m2 < 50:
        return 2
    elif m2 < 70:
        return 3
    elif m2 < 90:
        return 4
    elif m2 < 110:
        return 5
    else:
        return 6

In [14]:
stavbe2019['sobnost_ocenjena'] = stavbe2019['m2_neto'].apply(oceni_sobnost)

In [15]:
stavbe2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3137 entries, 0 to 3136
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ime_ko             3137 non-null   object        
 1   naselje            3127 non-null   object        
 2   letnica_izgradnje  3137 non-null   int64         
 3   m2_bruto           3137 non-null   float64       
 4   m2_neto            3137 non-null   float64       
 5   LETO               3137 non-null   datetime64[ns]
 6   cena               3137 non-null   float64       
 7   novogradnja        328 non-null    float64       
 8   novogradnja_1      3137 non-null   bool          
 9   sobnost_ocenjena   3137 non-null   int64         
dtypes: bool(1), datetime64[ns](1), float64(4), int64(2), object(2)
memory usage: 223.8+ KB


In [16]:
# Droping the original 'NOVOGRADNJA' column, as we now have 'NOVOGRADNJA_1' which has the same information in a boolean format.
stavbe2019.drop(columns=['novogradnja'], inplace=True)

In [18]:
# Uniforming the column names to lowercase
stavbe2019.columns = [col.lower() for col in stavbe2019.columns]

In [19]:
#uniforming the all values in all the columns to lowercase
for col in stavbe2019.columns:
    if stavbe2019[col].dtype == 'object':
        stavbe2019[col] = stavbe2019[col].str.lower()

In [20]:
# checking all the empty values in the 'naselje' column
stavbe2019[stavbe2019['naselje'].isna()]

,ime_ko,naselje,letnica_izgradnje,m2_bruto,m2_neto,leto,cena,novogradnja_1,sobnost_ocenjena
478,gameljne,NaN,1970,72.2,42.2,2019-01-01,70000.0,False,2
1875,udmat,NaN,1936,70.0,70.0,2019-01-01,40000.0,False,4
1919,poljansko predmestje,NaN,1895,64.2,39.0,2019-01-01,90000.0,False,2
2028,gameljne,NaN,2005,172.2,162.8,2019-01-01,145000.0,False,6
2276,trnovsko predmestje,NaN,1962,346.2,192.2,2019-01-01,700000.0,False,6
2561,spodnja šiška,NaN,1938,76.0,42.3,2019-01-01,70000.0,False,2
2802,gradišče ii,NaN,2017,332.5,254.2,2019-01-01,1410824.4,True,6
2815,gameljne,NaN,2005,172.2,162.8,2019-01-01,155000.0,False,6
2847,podgorica,NaN,1965,247.6,128.9,2019-01-01,215000.0,False,6
3112,glince,NaN,1984,276.0,276.0,2019-01-01,170000.0,False,6


In [21]:
# changing all the empty values in the 'naselje' column to 'ljubljana'
stavbe2019.fillna({'naselje': 'ljubljana'}, inplace=True)

In [22]:
stavbe2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3137 entries, 0 to 3136
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   ime_ko             3137 non-null   object        
 1   naselje            3137 non-null   object        
 2   letnica_izgradnje  3137 non-null   int64         
 3   m2_bruto           3137 non-null   float64       
 4   m2_neto            3137 non-null   float64       
 5   leto               3137 non-null   datetime64[ns]
 6   cena               3137 non-null   float64       
 7   novogradnja_1      3137 non-null   bool          
 8   sobnost_ocenjena   3137 non-null   int64         
dtypes: bool(1), datetime64[ns](1), float64(3), int64(2), object(2)
memory usage: 199.3+ KB


In [23]:
stavbe2019.to_csv(r"clean_GURS\stavbe2019_cleaned.csv", index=False, encoding='utf-8')